In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader=PyPDFLoader("C:\\Users\\Rudra\\Desktop\\RudraModi_InternshalaResume.pdf")
pdf_data=loader.load()
pdf_data

[Document(metadata={'source': 'C:\\Users\\Rudra\\Desktop\\RudraModi_InternshalaResume.pdf', 'page': 0}, page_content="Rudra Modirudramodi9560@gmail.com\n+91 9712193880\nVadodara\nEDUCATION Bachelor of Engineering (B.E), Computer Science &\nEngineering\nLD College Of Engineering\n2022 - 2026\nCGPA: 9.07/10\nPROJECTS Emoji prediction based on sentiment analysis\nMay 2024 - May 2024\nhttps://huggingface.co/R udra360/Emoji_Suggester\nIt's a NLP based sentiment analysis model for\nunderstanding the content of message without the\nconte xt of message, for giving suggestions into the\nmessage ar rived at notiﬁcation of any chatting app or\nweb..\nFace Detection Model Without T ransfer Learning\nMar 2024 - Mar 2024\nhttps://github.com/R udraModi360/Deep_L eaning_Basics/bl\nob/main/new_data_object_detection.ipynb\nA Deep L earning Based Model built using T ensorﬂow Used\nFor detecting F aces fr om\nimages built fr om Scratch W ithout T ransfer L earning\nGained T est_accuracy as 87.22%\nFlask A

In [2]:
groq_api_key = "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"

In [3]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
    "https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01",
    # bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("question js-question")),
)
web_data = loader.load()
web_data

[Document(metadata={'source': 'https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01'}, page_content='{"status":"success","data":{"candles":[["2024-07-23T00:00:00+05:30",1647.55,1647.8,1605,1618.4,26001076,0],["2024-07-22T00:00:00+05:30",1617,1651,1612.65,1642.55,26872411,0],["2024-07-19T00:00:00+05:30",1616,1623,1603.5,1607.3,13001076,0],["2024-07-18T00:00:00+05:30",1608,1623.95,1599.15,1614.8,16858532,0],["2024-07-16T00:00:00+05:30",1620.75,1627,1616.8,1619.75,14475162,0],["2024-07-15T00:00:00+05:30",1623.5,1629.8,1615.2,1622.1,16687847,0],["2024-07-12T00:00:00+05:30",1622,1638.4,1611.15,1622.7,28024980,0],["2024-07-11T00:00:00+05:30",1623.95,1625.85,1601,1621.9,22088537,0],["2024-07-10T00:00:00+05:30",1629.95,1640,1620.75,1626.1,22753591,0],["2024-07-09T00:00:00+05:30",1625,1646.7,1620.35,1636.5,24275439,0],["2024-07-08T00:00:00+05:30",1645.3,1654.95,1627.15,1635.35,19543900,0],["2024-07-05T00:00:00+05:30",1685,1685,1642.2,1648.1,41121274,0],["2

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
pdf_splitted=text_splitter.split_documents(pdf_data)
web_splitted=text_splitter.split_documents(web_data)
web_splitted

[Document(metadata={'source': 'https://api.upstox.com/v2/historical-candle/NSE_EQ%7CINE040A01034/day/2024-07-23/2024-07-01'}, page_content='{"status":"success","data":{"candles":[["2024-07-23T00:00:00+05:30",1647.55,1647.8,1605,1618.4,26001076,0],["2024-07-22T00:00:00+05:30",1617,1651,1612.65,1642.55,26872411,0],["2024-07-19T00:00:00+05:30",1616,1623,1603.5,1607.3,13001076,0],["2024-07-18T00:00:00+05:30",1608,1623.95,1599.15,1614.8,16858532,0],["2024-07-16T00:00:00+05:30",1620.75,1627,1616.8,1619.75,14475162,0],["2024-07-15T00:00:00+05:30",1623.5,1629.8,1615.2,1622.1,16687847,0],["2024-07-12T00:00:00+05:30",1622,1638.4,1611.15,1622.7,28024980,0],["2024-07-11T00:00:00+05:30",1623.95,1625.85,1601,1621.9,22088537,0],["2024-07-10T00:00:00+05:30",1629.95,1640,1620.75,1626.1,22753591,0],["2024-07-09T00:00:00+05:30",1625,1646.7,1620.35,1636.5,24275439,0],["2024-07-08T00:00:00+05:30",1645.3,1654.95,1627.15,1635.35,19543900,0],["2024-07-05T00:00:00+05:30",1685,1685,1642.2,1648.1,41121274,0],["2

In [5]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings_obj=OllamaEmbeddings(model='llava-phi3:3.8b')
embeddings=embeddings_obj.embed_documents(pdf_splitted)
embeddings

[[-0.2813180983066559,
  -0.344902366399765,
  0.1892853081226349,
  -0.5196676850318909,
  -1.5313620567321777,
  1.3926727771759033,
  -0.45429858565330505,
  -1.4812400341033936,
  1.078492283821106,
  -0.3064258098602295,
  0.6187928915023804,
  0.369424432516098,
  0.013802554458379745,
  -0.4820641875267029,
  1.0448631048202515,
  0.4219493269920349,
  0.2131832242012024,
  0.8362037539482117,
  -0.8292026519775391,
  -0.4282127320766449,
  -0.3545113801956177,
  0.5399909019470215,
  0.567368745803833,
  -0.6812769174575806,
  -1.148390531539917,
  0.453083336353302,
  -0.5797507166862488,
  -1.2988110780715942,
  0.6518299579620361,
  0.4631240963935852,
  0.7385134696960449,
  -0.5035187602043152,
  -0.21100088953971863,
  -0.3974657356739044,
  -0.15552499890327454,
  -0.34144437313079834,
  0.7291173338890076,
  1.5395662784576416,
  1.2666772603988647,
  -0.3290168344974518,
  -0.4299319386482239,
  0.6184267997741699,
  1.0683414936065674,
  -1.7535661458969116,
  -1.1587

In [7]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(pdf_splitted,embeddings_obj)
db

In [8]:
from langchain_community.vectorstores.chroma import Chroma
db_chroma=Chroma.from_documents(web_splitted,embeddings_obj)
db_chroma

In [19]:
query="what is the project for the candidate"
results=db_chroma.similarity_search(query)

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [10]:
results[0].page_content

'7-05T00:00:00+05:30",1685,1685,1642.2,1648.1,41121274,0],["2024-07-04T00:00:00+05:30",1759.75,1759.75,1724.85,1727.15,22924329,0],["2024-07-03T00:00:00+05:30",1791,1794,1764.65,1768.65,61608901,0],["2024-07-02T00:00:00+05:30",1715,1734.9,1702.75,1730.6,22960845,0],["2024-07-01T00:00:00+05:30",1680,1707.3,1680,1705.2,10593512,0]]}}'

In [11]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq

groq_api_key = "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"
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama-3.1-405b-reasoning")
prompt = ChatPromptTemplate.from_template(
    """
Answer the questions based on the provided context only.
Please provide the most accurate response based on the question
<context>
{context}
<context>
Questions:{input}

"""
)
document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain.chains import create_retrieval_chain
retriever = db_chroma.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [23]:
prompt="what can be next trend for 5 day onwards from today"

In [24]:
response=retrieval_chain.invoke(input={'input':prompt,
                        'context':web_splitted})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


In [25]:
print(response['answer'])

Based on the provided context, which appears to be historical stock market data, it's challenging to predict the next trend with certainty. However, I can provide some insights based on the available data.

The data shows a general downtrend in the stock price from July 3rd (1791) to July 23rd (1647.55). The last 5 days (July 16th to July 23rd) have shown a slight upward movement, but the overall trend remains bearish.

To make an educated guess about the next trend for the next 5 days, I'll consider the following factors:

1. The last 5 days have shown a slight upward movement, which could be a sign of a short-term reversal.
2. The overall trend is still bearish, and the stock price is still below the recent highs (July 3rd).
3. There is no clear indication of a strong reversal or a breakout above the recent highs.

Based on these factors, a possible next trend for the next 5 days could be:

* A continuation of the slight upward movement, potentially forming a small consolidation or a